# FFTW C2C F90 Sequential 576

(C2C = Complex to Complex)

Complex Multi-Dimensional DFTs 3D

    fftw_plan fftw_plan_dft_3d(int n0, int n1, int n2,
                   fftw_complex *in, fftw_complex *out,
                   int sign, unsigned flags);                          
- http://www.fftw.org/fftw3_doc/Complex-Multi_002dDimensional-DFTs.html

In [39]:
%%writefile fc2cs.f90
program main
    use, intrinsic :: iso_c_binding
    implicit none
    include "fftw3.f03"
    integer, parameter :: L = 576, M = 576, N = 576
    type(C_PTR) :: plan, cdata
    complex(C_DOUBLE_COMPLEX), pointer :: data(:,:,:)
    complex(C_DOUBLE_COMPLEX) :: s
    integer :: i, j, k
    double precision :: t0, t1, t2
    
    call cpu_time(t0)    ! time measurement
           
    ! in-place transform (note dimension reversal)
    cdata = fftw_alloc_complex(int(L * M * N, C_SIZE_T))
    call c_f_pointer(cdata, data, [L, M, N])

    ! create plan for in-place forward DFT (note dimension reversal)   
    plan = fftw_plan_dft_3d(N, M, L, data, data, &
                            FFTW_FORWARD, FFTW_ESTIMATE)

    ! fills the array with complex values
    do k = 1, N
        do j = 1, M
            do i = 1, L
                data(i, j, k) = sin( real(i + j + k) )
            enddo
        enddo
    enddo
    data = dcmplx( real(data) , 0 )
    
    call cpu_time(t1)    ! time measurement

    ! compute transform (as many times as desired)  
    call fftw_execute_dft(plan, data, data)   
    ! checksum
    s = sum(data)
    
    call cpu_time(t2)    ! time measurement
    
    call fftw_destroy_plan(plan)
    call fftw_free(cdata)
    
    ! result
    write(*, "('S: 'spf0.0spf0.0'j')", advance="no") s * 1e-5
    write(*, "(' | L: 'g0)", advance="no") L
    write(*, "(' | T1: 'sf0.4)", advance="no") t1-t0
    write(*, "(' | TF: 'sf0.4)", advance="no") t2-t1
    write(*, "(' | TT: 'sf0.4)") t2-t0
    
end

Overwriting fc2cs.f90


## Compile

- Compile using -O3 and uses system libraries
- Using: module load  mathlibs/fftw/3.3.8_openmpi-3.1_gnu

In [40]:
%%bash
export FFTW_ROOT=/scratch/app/mathlibs/fftw/3.3.8_openmpi-3.1_gnu
gfortran  -O3  -o fc2cs  fc2cs.f90  \
    -L $FFTW_ROOT/lib  -l fftw3  -l m  -I $FFTW_ROOT/include
ls fc2cs

fc2cs


## Check, imag = 0

In [41]:
! time ./fc2cs

S: +270.+0.j | L: 576 | T1: 7.8100 | TF: 16.6361 | TT: 24.4461

real	0m24.714s
user	0m21.812s
sys	0m2.896s


## Check, imag = real

(practically equal times)

In [38]:
! time ./fc2cs

S: +270.+270.j | L: 576 | T1: 7.8245 | TF: 16.4662 | TT: 24.2908

real	0m24.566s
user	0m21.517s
sys	0m3.042s


## Copy to /scratch

In [42]:
! cp fc2cs /scratch${PWD#"/prj"}

## Batch script

In [43]:
%%writefile fc2cs.srm
#!/bin/bash
#SBATCH --job-name fc2cs       # Job name
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:05:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Executable config
EXEC=$PWD/fc2cs

# Start
echo '-- run --------------------------------'
echo '$ srun -n' $SLURM_NTASKS ${EXEC##*/}
echo '-- output -----------------------------'
srun -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting fc2cs.srm


## Run

In [44]:
! sbatch fc2cs.srm
! sbatch fc2cs.srm
! sbatch fc2cs.srm

Submitted batch job 1330918
Submitted batch job 1330919
Submitted batch job 1330920


In [45]:
! squeue -n fc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1330918  cpu_small  PD  0:00     1    1
           1330919  cpu_small  PD  0:00     1    1
           1330920  cpu_small  PD  0:00     1    1


In [2]:
! squeue -n fc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


In [3]:
! cat /scratch${PWD#"/prj"}/slurm-1330918.out
! cat /scratch${PWD#"/prj"}/slurm-1330919.out
! cat /scratch${PWD#"/prj"}/slurm-1330920.out

- Job ID: 1330918
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1483
-- run --------------------------------
$ srun -n 1 fc2cs
-- output -----------------------------
S: +270.+0.j | L: 576 | T1: 5.9494 | TF: 13.3335 | TT: 19.2829
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1330919
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1483
-- run --------------------------------
$ srun -n 1 fc2cs
-- output -----------------------------
S: +270.+0.j | L: 576 | T1: 5.9561 | TF: 13.3401 | TT: 19.2963
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1330920
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1483
-- run --------------------------------
$ srun -n 1 fc2cs
-- output --------------------

### Version

In [4]:
! gfortran --version

GNU Fortran (GCC) 4.8.5 20150623 (Red Hat 4.8.5-36)
Copyright (C) 2015 Free Software Foundation, Inc.

GNU Fortran comes with NO WARRANTY, to the extent permitted by law.
You may redistribute copies of GNU Fortran
under the terms of the GNU General Public License.
For more information about these matters, see the file named COPYING



In [2]:
! module avail 2>&1 | grep -i fftw

mathlibs/fftw/3.3.8_intel
mathlibs/fftw/3.3.8_openmpi-2.0_gnu
mathlibs/fftw/3.3.8_openmpi-2.0_intel
mathlibs/fftw/3.3.8_openmpi-3.1_gnu


In [5]:
! hostnamectl | grep Operating

  Operating System: Red Hat Enterprise Linux Server 7.6 (Maipo)
